In [47]:
import numpy as np
import scipy.sparse as sp

In [48]:
def ensure_positive_definite(M, epsilon=1e-3, min_threshold=1e-6):
    """Ensure M is well-conditioned by adding a larger perturbation if needed."""
    min_eig = np.min(np.linalg.eigvals(M))
    
    if min_eig < min_threshold:
        print(f"Minimum eigenvalue too small ({min_eig:.2e}), adding {epsilon} to diagonal elements.")
        M += np.eye(M.shape[0]) * (abs(min_eig) + epsilon)
    
    return M

In [74]:
from enum import Enum, auto

class GraphType(Enum):
    STAR = auto()
    HUB = auto()
    AR2 = auto()
    RANDOM = auto()

In [49]:
def make_sparse_spd_matrix(
    n_dim=10,
    alpha=0.95,
    norm_diag=False,
    smallest_coef=0.1,
    largest_coef=0.9,
    random_state=None
):
    """
    Generate a sparse symmetric positive definite (SPD) matrix.

    Parameters
    ----------
    n_dim : int, default=10
        The size of the random matrix to generate.

    alpha : float, default=0.95
        The probability that a coefficient is zero, controlling sparsity.
        Higher values mean more sparsity. Should be between 0 and 1.

    norm_diag : bool, default=False
        If True, normalizes the matrix so that the diagonal elements are all 1.

    smallest_coef : float, default=0.1
        The smallest coefficient in the randomly generated values (between 0 and 1).

    largest_coef : float, default=0.9
        The largest coefficient in the randomly generated values (between 0 and 1).

    random_state : int or None, default=None
        Seed for random number generation, ensuring reproducible results.

    Returns
    -------
    ndarray or sparse matrix
        The generated sparse SPD matrix as a dense ndarray.
    """
    rng = np.random.default_rng(random_state)

    # Start with a negative identity matrix, which will form the basis of the Cholesky factor.
    chol = -sp.eye(n_dim, format="csc")

    # Generate a random sparse lower triangular matrix to add sparsity
    aux = sp.random(
        m=n_dim, n=n_dim, density=1 - alpha,
        data_rvs=lambda x: rng.uniform(low=smallest_coef, high=largest_coef, size=x),
        format="csc"
    )
    aux = sp.tril(aux, k=-1, format="csc")

    # Randomly permute rows and columns to avoid asymmetries
    permutation = rng.permutation(n_dim)
    aux = aux[permutation].T[permutation]

    # Add the sparse auxiliary matrix to the Cholesky factor
    chol += aux

    # Form the SPD matrix by taking the product of the Cholesky factor with its transpose
    prec = chol.T @ chol

    # Optionally normalize the diagonal to 1
    if norm_diag:
        d = sp.diags(1.0 / np.sqrt(prec.diagonal()))
        prec = d @ prec @ d
    prec = ensure_positive_definite(prec.toarray())

    return prec


In [50]:
import scipy.linalg as la
from scipy.linalg import solve_sylvester, norm

# Fix random number generator for reproducibility
np.random.seed(0)

In [51]:
def generate_sparse_covariance(n, N, alpha=0.8, random_state=42):
    """
    Generate a sparse inverse covariance matrix B, compute its associated covariance matrix E,
    and generate N samples from a multivariate normal distribution with covariance E.

    Parameters:
    - n (int): Dimension of the matrix.
    - N (int): Number of samples to generate.
    - alpha (float): Sparsity level for the sparse SPD matrix.
    - random_state (int): Random seed for reproducibility.

    Returns:
    - B (np.ndarray): Sparse inverse covariance matrix (precision matrix).
    - Strue (np.ndarray): True inverse covariance matrix (squared B).
    - E (np.ndarray): Covariance matrix (inverse of Strue).
    - y_samples (np.ndarray): Generated samples following N(0, E).
    - S (np.ndarray): Sample covariance matrix from the generated samples.
    """
    # Create sparse inverse covariance matrix (B)
    B = make_sparse_spd_matrix(n_dim=n, alpha=alpha, norm_diag=True, random_state=random_state)

    # Compute true inverse covariance matrix (Strue)
    Strue = np.linalg.matrix_power(B, 2)

    # Compute covariance matrix (E)
    E = np.linalg.inv(Strue)

    # Generate N samples Y ~ N(0, E)
    y_samples = la.sqrtm(E).dot(np.random.randn(n, N))

    # Calculate sample covariance matrix
    S = np.cov(y_samples)

    return B, Strue, E, y_samples, S


In [52]:
def newton_nare(A, B, C, D, X0, tol=1e-13, kmax=30):
    """
    Newton's method for solving the Nonlinear Algebraic Riccati Equation (NARE):
    C + XA + DX - XBX = 0
    """
    X = X0.copy()
    k = 0
    err = 1

    while err > tol and k < kmax:
        # Compute residual RX = C + XA + DX - XBX
        RX = C + X @ A + D @ X - X @ B @ X

        # Solve the Sylvester equation (D - XB)H + H(A - BX) = -RX for H
        H = solve_sylvester(D - X @ B, A - B @ X, -RX)

        # Update X
        X = X + H

        # Calculate the error
        err = norm(H, 1) / norm(X, 1)

        # Increment iteration counter
        k += 1

    # Check if the solution converged
    if k == kmax:
        print("Warning: reached the maximum number of iterations without convergence.")

    return X

In [53]:
# Soft thresholding function
def soft_thresholding(x, threshold):
    """Applies soft-thresholding elementwise."""
    return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

In [54]:

# ADMM Algorithm for Elastic-Net Penalized Precision Matrix Estimation
def admm_precision_matrix(S, lambda_, alpha, rho=1.0, max_iter=100, tol=1e-4):
    """
    ADMM algorithm for precision matrix estimation with elastic-net penalty.
    """
    n = S.shape[0]
    Z = np.zeros((n, n))
    Lambda = np.zeros((n, n))
    I = np.eye(n)  # Identity matrix

    # Initial B (can be initialized as identity matrix)
    B = np.eye(n)

    for k in range(max_iter):
        # Step 1: Update B using Newton NARE
        # Here, we set up the matrices to solve the NARE: A3 + XA1 + A4X - XA2X = 0
        A3 = - 2 * I
        A4 = Lambda - rho * Z
        A1 = 0 * I
        A2 = - (2 * S + rho * I)
        X0 = B  # Initial guess for Newton NARE

        # Solve for the new B using Newton NARE
        B_new = newton_nare(A1, A2, A3, A4, X0)

        # Step 2: Update Z elementwise using soft-thresholding
        Z_new = soft_thresholding(rho * B_new + Lambda, lambda_)
        Z_new = Z_new / rho

        # Step 3: Update Lambda (Lagrange multiplier)
        Lambda_new = Lambda + rho * (B_new - Z_new)

        # Check convergence
        if np.linalg.norm(B_new - B, ord='fro') < tol:
            print(f"ADMM Converged after {k+1} iterations.")
            break
        elif k == max_iter-1 :
            print(f"ADMM failed to converge after {k+1} iterations.")

        # Update for the next iteration
        B, Z, Lambda = B_new, Z_new, Lambda_new      

    return B

In [55]:
#Thresholding B_estimate
def hard_threshold(B_estimate,threshold):
  return np.where(np.abs(B_estimate) > threshold, B_estimate, 0)

Experimentation

In [56]:
import matplotlib.pyplot as plt
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [57]:
def evaluate_metrics_vs_lambda(lambda_values, n = 10, N= 100, alpha=1, rho=1.0, max_iter=100, tol=1e-4, threshold=1e-4, log_dir="experiment_logs"):
    B, _, E, _, S = generate_sparse_covariance(n, N)

    # Compute the minimum eigenvalue of B
    eigenvalues = np.linalg.eigvals(B)
    min_eigenvalue = np.min(eigenvalues)
    print("Minimum eigenvalue:", min_eigenvalue)

    metrics = {
        "lambda": [],
        "accuracy": [],
        "precision": [],
        "recall": [],
        "f1_score": [],
        "plot_paths": []
    }


    for lambda_ in lambda_values:
        # Compute estimated precision matrix
        B_estimate = admm_precision_matrix(B, lambda_, alpha, rho, max_iter, tol)
        
        # Apply thresholding
        B_estimate_thresholded = hard_threshold(B_estimate, threshold)

        # Plot sparsity patterns and save them
        plt.figure(figsize=(12, 12))

        plt.subplot(2, 2, 1)
        plt.spy(B)
        plt.title('B matrix', fontsize=16)

        # Placeholder for estimated matrix
        plt.subplot(2, 2, 2)
        plt.spy(B_estimate_thresholded)
        plt.title('B_hat (Estimated)', fontsize=16)

        plt.subplot(2, 2, 3)
        plt.spy(E)
        plt.title('True covariance', fontsize=16)

        plt.subplot(2, 2, 4)
        plt.spy(S)
        plt.title('Sample covariance', fontsize=16)

        plot_path = os.path.join(log_dir, f"sparsity_patterns_lambda_{lambda_:.3f}.png")
        plt.savefig(plot_path)
        plt.close()
        
        metrics["plot_paths"].append(plot_path)

        # Flatten for comparison
        ground_truth = (B != 0).astype(int).flatten()
        predicted = (B_estimate_thresholded != 0).astype(int).flatten()

        # Calculate metrics
        accuracy = accuracy_score(ground_truth, predicted)
        precision = precision_score(ground_truth, predicted)
        recall = recall_score(ground_truth, predicted)
        f1 = f1_score(ground_truth, predicted)

        # Store results
        metrics["lambda"].append(lambda_)
        metrics["accuracy"].append(accuracy)
        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
        metrics["f1_score"].append(f1)

    return metrics


In [58]:
# Plotting metrics vs. lambda
import matplotlib.pyplot as plt
import pandas as pd
import json
import numpy as np

In [60]:
n = 50
N = 5000

for _alpha in np.linspace(0, 1, 10):
    log_dir = f"experiment2_logs/n_{n}_N_{N}/alpha_{_alpha:.3f}"
    # Create a directory to save logs if it doesn’t exist
    os.makedirs(log_dir, exist_ok=True)

    # params
    args = {
        "lambda_values": np.linspace(0.1, 1, 20).tolist(),  # 20 values from 0.1 to 1.0
        "n": n,
        "N": N,
        "alpha": _alpha,
        "rho": 5.0,
        "max_iter": 200,
        "tol": 1e-4,
        "threshold": 1e-4,
        "log_dir": log_dir
    }
    print(f'Evaluating metrics for {args}')
    # Save args to a JSON file
    json_path = os.path.join(log_dir, "params.json")
    with open(json_path, "w") as json_file:
        json.dump(args, json_file, indent=4)

    metrics = evaluate_metrics_vs_lambda(**args)


    # Convert metrics to a DataFrame
    metrics_df = pd.DataFrame(metrics)

    # Save metrics
    excel_path = os.path.join(log_dir, "metrics.xlsx")
    metrics_df.to_csv(excel_path)
        
    # print("Metrics vs Lambda:")
    # print(metrics_df)

    for metric in ["accuracy", "precision", "recall", "f1_score"]:
        plt.plot(metrics["lambda"], metrics[metric], label=metric)
    plt.xscale("log")
    plt.xlabel("Lambda")
    plt.ylabel("Metric Value")
    plt.title("Metrics vs Lambda")
    plt.legend()
    # plt.show()

    plot_path = os.path.join(log_dir, f"metrics_vs_lambda.png")
    plt.savefig(plot_path)
    plt.close()

Evaluating metrics for {'lambda_values': [0.1, 0.1473684210526316, 0.19473684210526315, 0.24210526315789474, 0.2894736842105263, 0.33684210526315794, 0.38421052631578945, 0.43157894736842106, 0.4789473684210527, 0.5263157894736842, 0.5736842105263158, 0.6210526315789474, 0.6684210526315789, 0.7157894736842105, 0.7631578947368421, 0.8105263157894737, 0.8578947368421053, 0.9052631578947369, 0.9526315789473684, 1.0], 'n': 50, 'N': 5000, 'alpha': 0.0, 'rho': 5.0, 'max_iter': 200, 'tol': 0.0001, 'threshold': 0.0001, 'log_dir': 'experiment2_logs/n_50_N_5000/alpha_0.000'}
Minimum eigenvalue: 0.00010254377503318587
ADMM Converged after 148 iterations.
ADMM Converged after 92 iterations.
ADMM Converged after 65 iterations.
ADMM Converged after 52 iterations.
ADMM Converged after 43 iterations.
ADMM Converged after 38 iterations.
ADMM Converged after 34 iterations.
ADMM Converged after 31 iterations.
ADMM Converged after 29 iterations.
ADMM Converged after 27 iterations.
ADMM Converged after 25 

In [72]:
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def cross_validate_lambda(lambda_values, n=10, N=100, alpha=0.5, rho=1.0, max_iter=200, tol=1e-4, threshold=1e-4, cv_folds=5):
    """
    Perform k-fold cross-validation to find the optimal lambda for precision matrix estimation.
    
    Parameters:
    - lambda_values (list): Range of lambda values to test.
    - n (int): Dimension of the precision matrix.
    - N (int): Number of samples.
    - alpha (float): Sparsity level.
    - rho (float): ADMM parameter.
    - max_iter (int): Maximum ADMM iterations.
    - tol (float): Convergence tolerance.
    - threshold (float): Hard-thresholding value.
    - cv_folds (int): Number of cross-validation folds.

    Returns:
    - best_lambda (float): The optimal lambda based on F1-score.
    - cv_results (pd.DataFrame): DataFrame with lambda vs. performance metrics.
    """
    
    # Generate the full dataset
    B, _, _, y_samples, _ = generate_sparse_covariance(n, N, alpha=alpha)
    
    # Split data for cross-validation
    kf = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
    
    results = {
        "lambda": [],
        "mean_f1_score": [],
        "mean_precision": [],
        "mean_recall": []
    }

    for lambda_ in lambda_values:
        f1_scores = []
        precision_scores = []
        recall_scores = []
        
        for train_index, val_index in kf.split(y_samples.T):  # Split samples along columns
            y_train, y_val = y_samples[:, train_index], y_samples[:, val_index]
            
            # Compute sample covariance matrix from training set
            S_train = np.cov(y_train)

            # Estimate precision matrix
            B_estimate = admm_precision_matrix(S_train, lambda_, alpha, rho, max_iter, tol)
            B_estimate_thresholded = hard_threshold(B_estimate, threshold)
            
            # Flatten for comparison
            ground_truth = (B != 0).astype(int).flatten()
            predicted = (B_estimate_thresholded != 0).astype(int).flatten()
            
            # Compute metrics
            precision = precision_score(ground_truth, predicted)
            recall = recall_score(ground_truth, predicted)
            f1 = f1_score(ground_truth, predicted)

            precision_scores.append(precision)
            recall_scores.append(recall)
            f1_scores.append(f1)

        # Store mean performance across folds
        results["lambda"].append(lambda_)
        results["mean_f1_score"].append(np.mean(f1_scores))
        results["mean_precision"].append(np.mean(precision_scores))
        results["mean_recall"].append(np.mean(recall_scores))
    
    # Convert results to a DataFrame
    cv_results = pd.DataFrame(results)
    
    # Find best lambda (max F1-score)
    best_lambda = cv_results.loc[cv_results["mean_f1_score"].idxmax(), "lambda"]
    
    # Plot performance vs. lambda
    plt.figure(figsize=(8, 5))
    plt.plot(cv_results["lambda"], cv_results["mean_f1_score"], marker="o", label="F1-score")
    plt.plot(cv_results["lambda"], cv_results["mean_precision"], marker="s", linestyle="--", label="Precision")
    plt.plot(cv_results["lambda"], cv_results["mean_recall"], marker="d", linestyle="--", label="Recall")
    plt.xscale("log")
    plt.xlabel("Lambda")
    plt.ylabel("Score")
    plt.title("Cross-Validation Performance vs Lambda")
    plt.legend()
    plt.grid()
    plt.show()

    print(f"Best Lambda based on CV: {best_lambda:.4f}")

    return best_lambda, cv_results


In [73]:
lambda_values = np.linspace(0, 1, 20)  # Test lambda values from 0.001 to 10
best_lambda, cv_results = cross_validate_lambda(lambda_values, n=50, N=75)

Minimum eigenvalue too small (5.33e-09), adding 0.001 to diagonal elements.
ADMM failed to converge after 200 iterations.
ADMM failed to converge after 200 iterations.
ADMM failed to converge after 200 iterations.
ADMM failed to converge after 200 iterations.
ADMM failed to converge after 200 iterations.
ADMM Converged after 46 iterations.
ADMM Converged after 46 iterations.
ADMM Converged after 44 iterations.
ADMM Converged after 47 iterations.
ADMM Converged after 43 iterations.
ADMM Converged after 61 iterations.
ADMM Converged after 65 iterations.
ADMM Converged after 64 iterations.
ADMM Converged after 66 iterations.
ADMM Converged after 64 iterations.
ADMM Converged after 87 iterations.
ADMM Converged after 81 iterations.
ADMM Converged after 86 iterations.
ADMM Converged after 88 iterations.
ADMM Converged after 89 iterations.
ADMM Converged after 106 iterations.
ADMM Converged after 117 iterations.
ADMM Converged after 112 iterations.
ADMM Converged after 115 iterations.
ADMM C

KeyboardInterrupt: 